In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    # get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

plates = [
    "441_20230807","449_20230807","310_20230830"
]
directory_targ = "/projects/0/einf914/transport/"
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]


import pickle
import scipy.io as sio

path_root = f"/scratch-shared/amftrack/graph_stacks"

for plate_id in plates:
    # update_plate_info(directory_targ, local=True,strong_constraint=False)
    # all_folders = get_current_folders(directory_targ, local=True)
    folders = all_folders.loc[all_folders["unique_id"] == plate_id]
    folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
    folders = folders.sort_values(by="folder")
    
    folders = folders.sort_values(by="datetime")
    
    exp = Experiment(directory_targ)
    # i = indexes[plate_id_video]
    # i = np.where(folders['folder'] == indexes[plate_id_video])[0][0]
    # selection = folders[folders['folder'].isin(indexes.values())]
    for i in range(1,len(folders)):
        selection =folders.iloc[i : i + 1]
        folder_list = list(selection["folder"])
        directory_name = folder_list[0]
        path_snap = directory_targ + directory_name
        transform = sio.loadmat(path_snap + "/Analysis/transform_new.mat")
        exp.load(selection, suffix="_realigned")
        for t in range(exp.ts):
            exp.load_tile_information(t)
            exp.save_location = ""
            
            load_study_zone(exp)
        os.makedirs(os.path.join(path_root,plate_id,),exist_ok = True)
        graph = exp.nx_graph[0]
        graph_to_save = graph.copy()
        for u, v, data in graph_to_save.edges(data=True):
            data["pixels"] = data.pop("pixel_list")
        node_not_in_ROI = []
        for node in graph_to_save:
            if not is_in_ROI_node(Node(node, exp), 0):
                node_not_in_ROI.append(node)
        graph_to_save.remove_nodes_from(node_not_in_ROI)
        nx.set_node_attributes(graph_to_save, exp.positions[0], 'position')
        path_tot = os.path.join(path_root,plate_id,f"graph{i:03d}.pickle")
        pickle.dump((graph_to_save,transform,selection.iloc[0]), open(path_tot, 'wb'))
        break